In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, avg
import matplotlib.pyplot as plt
import time

spark_session = SparkSession \
    .builder \
    .master("spark://192.168.2.156:7077") \
    .appName("ProjectGR25") \
    .config("spark.eventLog.enabled", "false") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.dynamicAllocation.minExecutors", 1) \
    .config("spark.dynamicAllocation.maxExecutors", 1) \
    .config("spark.dynamicAllocation.initialExecutors", 1) \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
    .config("spark.shuffle.service.enabled", False) \
    .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
    .config("spark.executor.cores", 6) \
    .config("spark.driver.cores", 2) \
    .config("spark.driver.port", 9999) \
    .config("spark.blockManager.port", 10005) \
    .getOrCreate()

spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 12:11:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 12:11:45 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [2]:
# Read CSV from hdfs and compute time
import time
start = time.time()
df = spark_session.read.csv("hdfs://192.168.2.92:9000/spark-data/.csv", header=True, inferSchema=True)
final = time.time() - start
print(f"Time read:{final}")

# Print Data Frame and compute time
start = time.time()
df.show()
final = time.time() - start
print(f"Time read:{final}")

Time read:22.05821681022644
+--------------------+------------------+--------------------+--------------------+--------------------+-------+-------------------+---------+-------+---+----+--------+------------+------+----+
|           File Path|           Song ID|               Title|              Artist|             Release|  Genre|    Song Hotttnesss| Duration|  Tempo|Key|Mode|Loudness|Danceability|Energy|Year|
+--------------------+------------------+--------------------+--------------------+--------------------+-------+-------------------+---------+-------+---+----+--------+------------+------+----+
|MillionSongSubset...|SOBUDOC12A6D4F8AC2|After All The Lov...|               Alias|               Alias|Unknown|0.38293509007501925|256.67873|120.093|  0|   1| -10.223|         0.0|   0.0|   0|
|MillionSongSubset...|SOLMCFH12A58A7FACE|   Under The Weather|         KT Tunstall|Eye To The Telescope|Unknown| 0.6944535460616664| 216.5024|142.121|  7|   1|  -9.911|         0.0|   0.0|2004|
|M

In [3]:
# start time
start = time.time()

# Show missing values
missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
missing_values.show()

# Danceability 0 for the whole dataset
print(df.filter(df.Danceability == 0).count())

# Energy 0 for the whole dataset
print(df.filter(df.Energy == 0).count())

# Some of the years are missing
print(df.filter(df.Year == 0).count())

# Genre is Unknown for the whole dataset
print(df.filter(df.Genre == "Unknown").count())

# Final time
final = time.time() - start
print(f"Time read:{final}")

+---------+-------+-----+------+-------+-----+---------------+--------+-----+----+----+--------+------------+------+----+
|File Path|Song ID|Title|Artist|Release|Genre|Song Hotttnesss|Duration|Tempo| Key|Mode|Loudness|Danceability|Energy|Year|
+---------+-------+-----+------+-------+-----+---------------+--------+-----+----+----+--------+------------+------+----+
|        0|      0| 1267|     1|      1|    1|        5515866|    1268| 1268|1268|1268|    1268|        1268|  1268|1268|
+---------+-------+-----+------+-------+-----+---------------+--------+-----+----+----+--------+------------+------+----+



12671360


[Stage 9:=====================================================>   (16 + 1) / 17]

12671360


6741155


[Stage 15:====================================================>   (16 + 1) / 17]

12671360
Time read:42.49562668800354


In [4]:
# Drop rows that do not give any information 
df = df.drop("Genre", "Danceability", "Energy")
df.show(5)

[Stage 18:>                                                         (0 + 1) / 1]

+--------------------+------------------+--------------------+------------------+--------------------+-------------------+---------+-------+---+----+--------+----+
|           File Path|           Song ID|               Title|            Artist|             Release|    Song Hotttnesss| Duration|  Tempo|Key|Mode|Loudness|Year|
+--------------------+------------------+--------------------+------------------+--------------------+-------------------+---------+-------+---+----+--------+----+
|MillionSongSubset...|SOBUDOC12A6D4F8AC2|After All The Lov...|             Alias|               Alias|0.38293509007501925|256.67873|120.093|  0|   1| -10.223|   0|
|MillionSongSubset...|SOLMCFH12A58A7FACE|   Under The Weather|       KT Tunstall|Eye To The Telescope| 0.6944535460616664| 216.5024|142.121|  7|   1|  -9.911|2004|
|MillionSongSubset...|SOOUBMT12A8C13DB6D|                90er|        Headliners|  Das Album zum Film|               NULL|347.79383|170.041|  8|   1|  -4.281|2006|
|MillionSongSubs

In [1]:
# Plot and compute time
start = time.time()

# Select rows that have year values
filtered_year_df = df.filter(col("year") != 0)

# Group by year and compute average song length
trend_df = filtered_year_df.groupBy("year").agg(avg("duration").alias("average_length"))

# Pandas for plotting
trend_pd = trend_df.toPandas().sort_values(by="year")

# Plot mean times over the years
plt.figure(figsize=(12, 6))
plt.plot(trend_pd["year"], trend_pd["average_length"], marker='o', linestyle='-')
plt.xlabel("Year")
plt.ylabel("Average Song Length (seconds)")
plt.title("Average Song Length Over the Years")
plt.grid()
plt.show()

final = time.time() - start
print(f"Time read:{final}")

NameError: name 'time' is not defined

In [11]:
spark_session.stop()